In [1]:
from torch.utils.data import DataLoader
from streaming import StreamingDataset
import time
from tqdm import tqdm   
import torch
import streaming 

/root/miniconda3/envs/videos/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Option 2

In [ ]:
streaming.base.util.clean_stale_shared_memory()

In [20]:
# Remote directory (S3 or local filesystem) where dataset is stored
remote_dir = '/root/projects/rl-nlp/videos/data_downsampled/mds-split/00000'

# Local directory where dataset is cached during operation
local_dir = '/root/projects/rl-nlp/videos/cache/split'
dataset = StreamingDataset(local=local_dir, remote=remote_dir, split=None, shuffle=False)

# Create PyTorch DataLoader
dataloader_split = DataLoader(dataset)

In [21]:
start = time.time()
for i, data in enumerate(tqdm(dataloader_split)):
    break
end = time.time() - start
print(end)

  0%|          | 0/10 [00:04<?, ?it/s]

4.022639989852905


In [7]:
frame = data['frame']
subtitle = data['subtitle']
batch_size = frame.shape[0]
num_frames = frame.shape[1]
num_chunks = frame.shape[2]
frame.shape

torch.Size([1, 140, 5, 360, 640, 3])

In [8]:
subtitle.shape

torch.Size([1, 140, 5, 40])

# Option 1

In [ ]:
streaming.base.util.clean_stale_shared_memory()

In [24]:
# Remote directory (S3 or local filesystem) where dataset is stored
remote_dir = '/root/projects/rl-nlp/videos/data_downsampled/mds-not-split/00000'

# Local directory where dataset is cached during operation
local_dir = '/root/projects/rl-nlp/videos/cache/not-split'
dataset = StreamingDataset(local=local_dir, remote=remote_dir, split=None, shuffle=False)

# Create PyTorch DataLoader
dataloader_not_split = DataLoader(dataset)

In [25]:
start = time.time()
chunked_frames = []
chunked_subtitle = []
chunk_unit = 5

for i, data in enumerate(tqdm(dataloader_not_split)):
    num_chunks, remainder = divmod(data['frame'].shape[1], chunk_unit)
    chunked_frame = torch.chunk(data['frame'][:, :-remainder, :, :, :], num_chunks, dim=1)
    chunked_subtitle = torch.chunk(data['subtitle'][:, :-remainder, :], num_chunks, dim=1)
    break
end = time.time() - start
print(end)

  0%|          | 0/10 [00:04<?, ?it/s]

4.6408374309539795


In [26]:
data['frame'].shape

torch.Size([1, 703, 360, 640, 3])